In [1]:
import os
import requests
import openai
import webvtt
from langchain.text_splitter import CharacterTextSplitter
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.docstore.document import Document
from langchain.vectorstores.faiss import FAISS
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.chains.question_answering import load_qa_chain

from langchain.document_loaders import TextLoader

Load agenda items and transcript

In [2]:
transcript_dir = "example_22min_transcript.txt"
agenda_dir = "example_22min_agenda.txt"

with open(transcript_dir, "r") as file:
    transcript_text = file.read()
    
with open(agenda_dir, "r") as file:
    agenda_text = file.read()

In [4]:
agenda_text

'Duke and Duchess of Sussex asked to vacate UK home Frogmore Cottage.\nAlex Muradugh guilty.\nData Centers Intro/definition and origins/size, density and efficiency growth.'

Using langchain to break our meeting transcript into chunks and call OpenAI's API.

In [6]:
API_KEY = os.environ['OPENAI_API_KEY']

In [21]:
from langchain.chat_models import ChatOpenAI
llm = ChatOpenAI(model_name="gpt-3.5-turbo-0301", openai_api_key=API_KEY)

source_chunks = []
splitter = RecursiveCharacterTextSplitter()
for chunk in splitter.split_text(transcript_text):
    source_chunks.append(Document(page_content=chunk))
search_index = FAISS.from_documents(source_chunks, OpenAIEmbeddings(openai_api_key = API_KEY))

chain = load_qa_chain(llm)

def return_llm_answer(question):
    return(
        chain(
            {
                "input_documents": search_index.similarity_search(question, k=4),
                "question": question,
            },
            return_only_outputs=True,
        )["output_text"]
    )

In [32]:
overall_summary = return_llm_answer("Provide a summary of the meeting.")

In [33]:
overall_summary

"The meeting started with everyone discussing how their week went, and Jiahe Feng mentioned being busy with midterms and excited for spring break. They then discussed the news of the Duke and Duchess of Sussex being asked to vacate Frogmore College, a royal residence gifted to them by Queen Elizabeth II, and the controversy over the cost of renovation. \n\nThey also discussed the guilty verdict of lawyer Alex Murdaugh for the murder of his wife and son, as well as his embezzlement of clients' funds. The group agreed to prepare a summary of his charges and verdicts for the next meeting. \n\nThey then discussed data centers, their modular design, and their role in cloud computing, as well as the historical evolution of equipment racks. The group agreed to watch a show on data centers and share their thoughts at the next meeting. Overall, the meeting covered a range of topics, from current events and legal issues to technical discussions on data centers."

In [36]:
agenda_summaries = []
for agenda in agenda_text.split("\n"):
    agenda_summaries.append(return_llm_answer("From this meeting, provide a summary related to this agenda item" + agenda))

In [35]:
agenda_summaries

["During the meeting, it was discussed that the Duke and Duchess of Sussex had been asked to leave their official UK residence, Frogmore Cottage. The property, gifted to them by the late Queen Elizabeth II, is owned by the Crown Estate and is located in Windsor's estate west of London. The couple had refurbished the cottage and made it their UK base, but it was later revealed that around $3 million of British taxpayers' money had been used for the renovation. Following their move to California, they paid back the public money. The news was reportedly confirmed by a spokesman for the couple, and Buckingham Palace declined to comment on the matter. It was noted that any discussions related to the matter would be a private family matter.",
 "During the meeting, the topic of discussion was Alex Murdaugh, a first-generation lawyer from South Carolina who was recently convicted of killing his wife and son. The discussion was initiated with a request for a summary of Murdaugh's charges and ve

In [39]:
agenda_text

'Duke and Duchess of Sussex asked to vacate UK home Frogmore Cottage.\nAlex Muradugh guilty.\nData Centers Intro/definition and origins/size, density and efficiency growth.'

In [40]:
return_llm_answer("Provide detailed summary that is not included from these agenda items:" + agenda_text)

'There is no additional context provided beyond these agenda items to include in a summary.'